In [1]:
import theano
from theano import tensor as T
import numpy as np
import sys
sys.path.append("../lib")
from load import mnist

Using gpu device 0: GeForce GTX 980


Simple Compiled function example

In [3]:
a = T.scalar()
b = T.scalar()

y = a * b

multiply = theano.function(inputs=[a, b], outputs=y)

print multiply(1, 2) #2
print multiply(3, 3) #9

2.0
9.0


In [4]:
trX = np.linspace(-1, 1, 101)
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33
print trY.shape

X = T.scalar()
print X
Y = T.scalar()
print Y


(101,)
<TensorType(float32, scalar)>
<TensorType(float32, scalar)>


In [5]:
def model1(X, w):
    return X * w

def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def model2(X, w):
    return T.nnet.softmax(T.dot(X, w))

In [6]:
w = theano.shared(np.asarray(0., dtype=theano.config.floatX))
y = model1(X, w)

cost = T.mean(T.sqr(y - Y))
gradient = T.grad(cost=cost, wrt=w)
updates = [[w, w - gradient * 0.01]]

train = theano.function(inputs=[X, Y], outputs=cost, updates=updates, allow_input_downcast=True)


In [7]:
for i in range(100):
    for x, y in zip(trX, trY):
        train(x, y)
        
print w.get_value() #something around 2

2.04096889496


Logistic Regression

In [9]:
trX, teX, trY, teY = mnist(onehot=True)

X = T.fmatrix()
Y = T.fmatrix()

w = init_weights((784, 10))

py_x = model2(X, w)
y_pred = T.argmax(py_x, axis=1)

cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * 0.05]]

In [10]:
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

In [ ]:
for i in range(100):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        a = train(trX[start:end], trY[start:end])
    print i, np.mean(np.argmax(teY, axis=1) == predict(teX))

0 0.925
1 0.9251
2 0.9251
3 0.9251
4 0.9249
5 0.9249
6 0.9249
7 0.9248
8 0.9248
9 0.9247
10 0.9248
11 0.9247
12 0.9247
13 0.9248
14 0.9247
15 0.9247
16 0.9247
17 0.9247
18 0.9248
19 0.9249
20 0.925
21 0.925
22 0.9251
23 0.9251
24 0.9251
25 0.9251
26 0.9251
27 0.9251
28 0.9252
29 0.925
30 0.9251
31 0.9251
32 0.9251
33 0.9251
34 0.925
35 0.925
36 0.9251
37